In [ ]:
import sys
import os

parent = os.path.dirname(os.path.realpath("../"))
sys.path.append(parent)

import numpy as np
import scipy
import tqdm
import open3d as o3d
import matplotlib.pyplot as plt
import glob

from core import *
from utils import phantom_builder
from utils import geometry
from utils import utils

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

overwrite_figures = True

In [ ]:
test_phantom = phantom.Phantom(
    voxel_dims=(0.25e-3, 0.25e-3, 0.25e-3),
    matrix_dims=(256, 256, 128),
    baseline=(1500, 1000),
    seed=5678,
)

water = tissue.Tissue(name="water", c=1500, rho=1000, sigma=0, scale=0.001, label=0)
wire = tissue.Tissue(name="wire", c=1600, rho=1100, sigma=30, scale=0.0001, label=1)

pos = 0.016
test_phantom.add_tissue(water, mask=np.ones(test_phantom.matrix_dims))

test_phantom.add_tissue_sphere((0, 0, 0), 8, wire)
test_phantom.add_tissue_sphere((pos, 0, 0), 16, wire)
test_phantom.add_tissue_sphere((0, pos, 0), 8, wire)
test_phantom.add_tissue_sphere((pos, pos, 0), 4, wire)

test_phantom.set_default_tissue("water")
test = test_phantom.get_complete()

In [ ]:
plt.imshow(test[0, :, :, test_phantom.matrix_dims[2] // 2])

plt.gca().set_xticks(np.linspace(0, test_phantom.matrix_dims[1], 9))
plt.gca().set_yticks(np.linspace(0, test_phantom.matrix_dims[0], 9))
plt.gca().grid(color="w", linestyle="-", linewidth=0.1)

In [ ]:
num_transducers = 2
transducers = [
    transducer.Planewave(
        max_frequency=2e6,
        elements=256,
        width=36e-3,
        height=10e-3,
        sensor_sampling_scheme="not_centroid",
        sweep=np.pi / 4,
        ray_num=5,
        imaging_ndims=2,
        focus_elevation=50e-3,
    )
    for i in range(num_transducers)
]

for t in transducers:
    t.make_sensor_coords(test_phantom.baseline[0])

In [ ]:
cont_transducer_set = transducer_set.TransducerSet(transducers, seed=8888)
cont_transducer_set.assign_pose(0, geometry.Transform([0, 0, 0], [-0.02, 0, 0]))
cont_transducer_set.assign_pose(1, geometry.Transform([np.pi / 2, 0, 0], [0, -0.02, 0]))


cont_transducer_set.plot_transducer_fovs(scale=0.1)
cont_transducer_set.plot_transducer_coords(scale=0.1)

In [ ]:
extended_sensor = sensor.Sensor(
    transducer_set=cont_transducer_set, aperture_type="extended_aperture"
)
tar_sensor = sensor.Sensor(
    transducer_set=cont_transducer_set, aperture_type="transmit_as_receive"
)

In [ ]:
simprops = simulation.SimProperties(
    grid_size=(50e-3, 50e-3, 20e-3),
    PML_size=(16, 8, 8),
    PML_alpha=2,
    t_end=12e-5,  # [s]
    bona=6,  # parameter b/a determining degree of nonlinear acoustic effects
    alpha_coeff=0.5,  # [dB/(MHz^y cm)]
    alpha_power=1.5,
)

In [ ]:
extended_experiment = experiment.Experiment(
    simulation_path="extended_aperture_spheres",
    sim_properties=simprops,
    phantom=test_phantom,
    transducer_set=cont_transducer_set,
    sensor=extended_sensor,
    nodes=1,
    results=None,
    indices=None,
    workers=2,
    additional_keys=[],
)

tar_experiment = experiment.Experiment(
    simulation_path="tar_aperture_spheres",
    sim_properties=simprops,
    phantom=test_phantom,
    transducer_set=cont_transducer_set,
    sensor=tar_sensor,
    nodes=1,
    results=None,
    indices=None,
    workers=2,
    additional_keys=[],
)

extended_experiment.save()
tar_experiment.save()

In [ ]:
gm = extended_experiment.visualize_sensor_mask(
    index=[slice(0, -1, 1), slice(0, -1, 1), test_phantom.matrix_dims[2] // 2 + 1]
)
plt.gca().set_title("aperture")
plt.gca().set_xticks([-1, 256])
plt.gca().set_xticklabels(["0", "0.064"])
plt.gca().set_yticks([0, 256])
plt.gca().set_yticklabels(["0.064", "0"])
plt.savefig("spheres_phantom.png", dpi=300, bbox_inches="tight")

In [ ]:
extended_experiment.plot_ray_path(0)

In [ ]:
test_experiment = experiment.Experiment.load("tar_aperture_spheres")
test_experiment.run(dry=True)
test_experiment.run(repeat=True)

In [ ]:
test_experiment = experiment.Experiment.load("extended_aperture_spheres")
test_experiment.run(dry=True)
test_experiment.run(repeat=True)

# Reconstruction

In [ ]:
test_experiment = experiment.Experiment.load("tar_aperture_spheres")
test_experiment.run(dry=True)
test_experiment.add_results()
test_reconstruction = reconstruction.Compounding(experiment=test_experiment)

In [ ]:
images1 = test_reconstruction.compound(
    workers=1, resolution_multiplier=2, local=True, combine=False
)

In [4]:
image1 = np.sum(images1, axis=0) / len(images1)
image1t1 = np.sum(images1[: len(images1) // 2], axis=0) / (len(images1) // 2)
image1t2 = np.sum(images1[len(images1) // 2 :], axis=0) / (len(images1) // 2)
image1_mult = np.multiply(image1t1, image1t2)

In [ ]:
from scipy.interpolate import NearestNDInterpolator

sensor = test_experiment.get_sensor_mask()
x = []
y = []
z = []
for i in tqdm.tqdm(range(sensor.shape[0])):
    for j in range(sensor.shape[1]):
        x.append(i)
        y.append(j)
        z.append(sensor[i, j, :].sum(axis=-1))
X = np.linspace(0, sensor.shape[0], image1.shape[0])
Y = np.linspace(0, sensor.shape[1], image1.shape[1])
X, Y = np.meshgrid(X, Y, indexing="ij")  # 2D grid for interpolation
interp = NearestNDInterpolator(list(zip(x, y)), z)
Z = interp(X, Y)
sensor_mask = Z
sensor_mask1 = sensor_mask.copy()
sensor_mask1[65:] = 0
sensor_mask2 = sensor_mask.copy()
sensor_mask2[:, 65:] = 0

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(2.5, 5))
contour_cmap = plt.cm.get_cmap("plasma")

psf = 20 * np.log10(image1t1 / 1000000)
image = psf[:, :, psf.shape[2] // 2]

im = ax[0].imshow(image, cmap="gray_r", vmin=-100, vmax=-50)
ax[0].imshow(sensor_mask1, cmap="cool_r", alpha=(sensor_mask1 > 0) * 1.0)
CS = ax[0].contour(
    image, cmap=contour_cmap, linewidths=0.5, levels=[-90, -85, -80, -75, -70, -65, -60]
)

ax[0].set_title("sequential t1")

psf = 20 * np.log10(image1t2 / 1000000)
image = psf[:, :, psf.shape[2] // 2]

im = ax[1].imshow(image, cmap="gray_r", vmin=-100, vmax=-50)
ax[1].imshow(sensor_mask2, cmap="cool_r", alpha=(sensor_mask2 > 0) * 1.0)
CS = ax[1].contour(
    image, cmap=contour_cmap, linewidths=0.5, levels=[-90, -85, -80, -75, -70, -65, -60]
)

ax[1].set_title("sequential t2")

# plt.colorbar(im, cax=ax.inset_axes([1.035, 0, 0.04, 1]), label='dB', ticks=[-100,-75,-70,-65,-60,-55,-50,0])
# proxy = [plt.Rectangle((0,0),1,1,fc = pc.get_edgecolor()[0]) for pc in CS.collections]
# plt.legend(proxy[::-1], ['','','','','',''][::-1], frameon=False, handlelength=1, handleheight=0.01, fontsize=10, bbox_to_anchor=(1.138, 0.55), labelspacing = 0.33)
# ax.clabel(CS, inline=True, fontsize=10)

ax[0].set_xticks([])
ax[0].set_yticks([])
ax[1].set_xticks([])
ax[1].set_yticks([])
if overwrite_figures:
    plt.savefig(
        "sequential_aperture_individual_spheres.png", dpi=300, bbox_inches="tight"
    )

In [ ]:
psf = 20 * np.log10(image1 / 1000000)
image = psf[:, :, psf.shape[2] // 2]

fig, ax = plt.subplots()
contour_cmap = plt.cm.get_cmap("plasma")

im = ax.imshow(image, cmap="gray_r", vmin=-100, vmax=-50)
ax.imshow(sensor_mask, cmap="cool_r", alpha=(sensor_mask > 0) * 1.0)
CS = ax.contour(
    image, cmap=contour_cmap, linewidths=0.5, levels=[-90, -85, -80, -75, -70, -65, -60]
)

ax.set_title("sequential averaged")

# plt.colorbar(im, cax=ax.inset_axes([1.035, 0, 0.04, 1]), label='dB', ticks=[-100,-75,-70,-65,-60,-55,-50,0])
# proxy = [plt.Rectangle((0,0),1,1,fc = pc.get_edgecolor()[0]) for pc in CS.collections]
# plt.legend(proxy[::-1], ['','','','','',''][::-1], frameon=False, handlelength=1, handleheight=0.01, fontsize=10, bbox_to_anchor=(1.138, 0.55), labelspacing = 0.33)
# ax.clabel(CS, inline=True, fontsize=10)

ax.set_xticks([])
ax.set_yticks([])
if overwrite_figures:
    plt.savefig("sequential_aperture_spheres.png", dpi=300, bbox_inches="tight")

In [ ]:
test_experiment = experiment.Experiment.load("extended_aperture_spheres")
test_experiment.run(dry=True)
test_experiment.add_results()
test_reconstruction = reconstruction.Compounding(experiment=test_experiment)

In [ ]:
images2 = test_reconstruction.compound(
    workers=1, resolution_multiplier=2, local=True, combine=False
)

In [10]:
image2 = np.sum(images2, axis=0) / len(images2)
image2t1 = np.sum(images2[: len(images2) // 2], axis=0) / (len(images2) // 2)
image2t2 = np.sum(images2[len(images2) // 2 :], axis=0) / (len(images2) // 2)
image2_mult = np.multiply(image2t1, image2t2)

In [ ]:
from scipy.interpolate import NearestNDInterpolator

sensor = test_experiment.get_sensor_mask()
x = []
y = []
z = []
for i in tqdm.tqdm(range(sensor.shape[0])):
    for j in range(sensor.shape[1]):
        x.append(i)
        y.append(j)
        z.append(sensor[i, j, :].sum(axis=-1))
X = np.linspace(0, sensor.shape[0], image2.shape[0])
Y = np.linspace(0, sensor.shape[1], image2.shape[1])
X, Y = np.meshgrid(X, Y, indexing="ij")  # 2D grid for interpolation
interp = NearestNDInterpolator(list(zip(x, y)), z)
Z = interp(X, Y)
sensor_mask = Z
sensor_mask1 = sensor_mask.copy()
sensor_mask1[65:] = 0
sensor_mask2 = sensor_mask.copy()
sensor_mask2[:, 65:] = 0

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(2.5, 5))
contour_cmap = plt.cm.get_cmap("plasma")

psf = 20 * np.log10(image2t1 / 1000000)
image = psf[:, :, psf.shape[2] // 2]

im = ax[0].imshow(image, cmap="gray_r", vmin=-100, vmax=-50)
ax[0].imshow(sensor_mask1, cmap="cool_r", alpha=(sensor_mask1 > 0) * 1.0)
CS = ax[0].contour(
    image, cmap=contour_cmap, linewidths=0.5, levels=[-90, -85, -80, -75, -70, -65, -60]
)

ax[0].set_title("synchronous t1")

psf = 20 * np.log10(image2t2 / 1000000)
image = psf[:, :, psf.shape[2] // 2]

im = ax[1].imshow(image, cmap="gray_r", vmin=-100, vmax=-50)
ax[1].imshow(sensor_mask2, cmap="cool_r", alpha=(sensor_mask2 > 0) * 1.0)
CS = ax[1].contour(
    image, cmap=contour_cmap, linewidths=0.5, levels=[-90, -85, -80, -75, -70, -65, -60]
)

ax[1].set_title("synchronous t2")

# plt.colorbar(im, cax=ax.inset_axes([1.035, 0, 0.04, 1]), label='dB', ticks=[-100,-75,-70,-65,-60,-55,-50,0])
# proxy = [plt.Rectangle((0,0),1,1,fc = pc.get_edgecolor()[0]) for pc in CS.collections]
# plt.legend(proxy[::-1], ['','','','','',''][::-1], frameon=False, handlelength=1, handleheight=0.01, fontsize=10, bbox_to_anchor=(1.138, 0.55), labelspacing = 0.33)
# ax.clabel(CS, inline=True, fontsize=10)

ax[0].set_xticks([])
ax[0].set_yticks([])
ax[1].set_xticks([])
ax[1].set_yticks([])
if overwrite_figures:
    plt.savefig(
        "synchronous_aperture_individual_spheres.png", dpi=300, bbox_inches="tight"
    )

In [ ]:
psf = 20 * np.log10(image2 / 1000000)
image = psf[:, :, psf.shape[2] // 2]

fig, ax = plt.subplots()
contour_cmap = plt.cm.get_cmap("plasma")

im = ax.imshow(image, cmap="gray_r", vmin=-100, vmax=-50)
ax.imshow(sensor_mask, cmap="cool_r", alpha=(sensor_mask > 0) * 1.0)
CS = ax.contour(
    image, cmap=contour_cmap, linewidths=0.5, levels=[-90, -85, -80, -75, -70, -65, -60]
)

ax.set_title("synchronous averaged")

plt.colorbar(
    im,
    cax=ax.inset_axes([1.035, 0, 0.04, 1]),
    label="dB",
    ticks=[-100, -90, -85, -80, -75, -70, -65, -60, -50],
)
proxy = [plt.Rectangle((0, 0), 1, 1, fc=pc.get_edgecolor()[0]) for pc in CS.collections]
plt.legend(
    proxy[::-1],
    ["", "", "", "", "", "", "", ""][::-1],
    frameon=False,
    handlelength=1,
    handleheight=0.01,
    fontsize=10,
    bbox_to_anchor=(1.138, 0.85),
    labelspacing=1.66,
)
# ax.clabel(CS, inline=True, fontsize=10)

ax.set_xticks([])
ax.set_yticks([])
if overwrite_figures:
    plt.savefig("synchronous_aperture_spheres.png", dpi=300, bbox_inches="tight")